<center><h1>Bitcoin Unlimit MultiSig Algorithm</h1></center>

In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from zkp_playground.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)

from operator import add
G = CG.G

import random

N = 0xFFFFF
random_f = lambda: CF(random.randint(1, N) % CF.P)


## Current Bitcoin Sig

In [3]:
priv = random_f()

In [4]:
from zkp_playground.crypto.ecdsa.secp256k1 import sign, verify

In [5]:
verify(G@priv, sign(priv, "test"), "test")

True

#### Proof

$x$ is privKey, $x, k, m \in \mathbb{Z}_p$

$$
k \leftarrow \mathbb{Z}_p\\
R = kG\\
s = \frac{m + xR_x}{k}\\
(R_x, s)
$$

$$
k = \frac{m + R_xx}{s}
$$

$(R_x, s)$ is the signature of message $m$

#### Verify

$$
V = \frac{m}{s}G + \frac{R_x}{s}xG \\
V=\frac{m+R_xx}{s}G\\
R_x =? V_x
$$



## The Unlimit MultiSig

### Proof:

$$
k \leftarrow \mathbb{Z}_p\\
R = kG\\
s = \frac{m + xR_x}{k}\\
(R_x, s)
$$

$$
\\s = \frac{n*m+\sum_{i=0}^n x_iR_{x}^i}{\sum_{i=0}^n k}
$$

In [6]:
from functools import reduce
from operator import add

In [7]:
n = 100
privLst = [random_f() for i in range(n)]

In [8]:
privkey = reduce(add, privLst)

In [9]:
pubkey = G @ privkey

In [10]:
reduce(add, [G@p for p in privLst]) == pubkey

True

In [11]:
from zkp_playground import bitcoin
from hashlib import sha256
from zkp_playground.utils import to_sha256int

In [12]:
gen_k = lambda x, n: CF(to_sha256int(str(x) + str(n)))

In [13]:
n = 10086
ks = [gen_k(p, n) for p in privLst]
sum_k = reduce(add, ks)
R = reduce(add, [G^i for i in ks])

In [14]:
sum_top = reduce(add, [CF(R.value[0]) * CF(p) for p in privLst])

In [15]:
m = CF(to_sha256int("hello priv"))

In [16]:
s = (m + sum_top) / sum_k

### Verify:


In [17]:
verify(pubkey, (CF(R.value[0]), s), "hello priv")

True